# Deep Fruit Vision Evaluation

Our detection, ripeness, and defect models are all trained and tested on different datasets. This notebook evaluates the overall performance of Deep Fruit Vision on a unique, hand-labelled test dataset.

In [1]:
import os
import shutil

from tqdm.notebook import tqdm
import torch
from torch.utils.data import Subset
from deepfruitvision import DeepFruitVision
from modules.datasets import EnsembleDataset, save_dataset
from torchvision.ops import box_iou

import matplotlib
from matplotlib import pyplot as plt

import numpy as np

%matplotlib inline

In [2]:
best_classification_weights = os.path.join('weights', 'detection', 'best_classification.pt')
best_detection_weights = os.path.join('weights', 'detection', 'best_detection.pt')
ripeness_weights = os.path.join('weights', 'ripeness', 'ripeness_model_fine_tuned')
defect_weights = os.path.join('weights', 'defect', 'defect_model_fine_tuned')

We can use the EnsembleDataset class to easily load images and labels from the test dataset. We reserve a bit of the ensemble dataset for fine-tuning our models. We use the seed to make sure that we get the same images across runs.

In [3]:
eval_dir = os.path.join('dataset', 'deepfruitvision_eval')
detection_eval_img_dir = os.path.join(eval_dir, 'images')
detection_eval_label_dir = os.path.join(eval_dir, 'labels')

os.makedirs(detection_eval_img_dir, exist_ok=True)
os.makedirs(detection_eval_label_dir, exist_ok=True)

seed = 123
np.random.seed(seed)
num_fine_tune_samples = 50

In [4]:
ensemble_dataset = EnsembleDataset('dataset', for_yolov5=True)

random_indices = np.random.permutation(len(ensemble_dataset))
fine_tune_indices = random_indices[:num_fine_tune_samples]
test_indices = random_indices[num_fine_tune_samples:]

In [5]:
yolov5_test_dataset = Subset(ensemble_dataset, test_indices)
save_dataset(yolov5_test_dataset, detection_eval_img_dir, detection_eval_label_dir) # save the test split of the ensemble dataset with Yolo-v5 labels to the disk

ensemble_dataset.for_yolov5 = False
deepfruitvision_test_dataset = Subset(ensemble_dataset, test_indices) # then just get the test split of the ensemble dataset

Saving dataset to dataset\deepfruitvision_eval\images and dataset\deepfruitvision_eval\labels: 100%|██████████| 105/105 [00:00<00:00, 145.52it/s]


## Detection mAP

The detection/Yolo-v5 model is responsible for detecting and classifying fruits, so we can just use Yolo-v5's built-in evaluation script. We've provided two trained Yolo-v5 models. The first model, `best_detection_weights.pt` was trained on the detection dataset. The second model, `best_classification_weights.pt` was trained on the detection dataset and fine-tuned on images from the ensemble dataset. The `best_detection_weights.pt` model is good at detecting fruits, but it can have issues classifying them if the fruit is in a different environment than one it saw in training (such as the ensemble dataset). It is a good model to use as a base for fine-tuning.

The following two tests are each Yolo-v5 model on the detection and classification task.

In [6]:
%run yolov5/val.py --data fine_tune_apple_papaya_mango.yaml --weights {best_detection_weights} --img 416 --task test

val: data=fine_tune_apple_papaya_mango.yaml, weights=['weights\\detection\\best_detection.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5\runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  2022-11-21 Python-3.10.5 torch-1.12.0 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\dataset\deepfruitvision_eval\labels... 105 images, 0 backgrounds, 0 corrupt: 100%|██████████| 105/105 00:05
test: New cache created: C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\dataset\deepfruitvision_eval\labels.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 4/4 00:03
           

In [4]:
%run yolov5/val.py --data fine_tune_apple_papaya_mango.yaml --weights {best_classification_weights} --img 416 --task test

val: data=fine_tune_apple_papaya_mango.yaml, weights=['weights\\detection\\best_classification.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5\runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  2022-11-21 Python-3.10.5 torch-1.12.0 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\dataset\deepfruitvision_eval\labels.cache... 105 images, 0 backgrounds, 0 corrupt: 100%|██████████| 105/105 00:00
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 4/4 00:03
                   all        105        417       0.85      0.815        0.9      0.667
                 apple        105  

The `best_classification_weights.pt` model is much better at classifying the fruit in the ensemble dataset. The `best_detection_weights.pt` model struggles on the ensemble dataset, but that's because it it is bad at classifying the fruit in the dataset since there is a large distribution shift from the detection dataset. However, when we only focus on detection, we see that the `best_detection_weights.pt` model has a competitive mAP and generalizes well to the ensemble dataset, despite not having seen any fruit from it. The `best_classification_weights.pt` model still does better, but that' due to the fine-tuning.

In [3]:
%run yolov5/val.py --data fine_tune_apple_papaya_mango.yaml --weights {best_detection_weights} --img 416 --task test --single-cls

val: data=fine_tune_apple_papaya_mango.yaml, weights=['weights\\detection\\best_detection.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=True, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5\runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  2022-11-21 Python-3.10.5 torch-1.12.0 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\dataset\deepfruitvision_eval\labels.cache... 105 images, 0 backgrounds, 0 corrupt: 100%|██████████| 105/105 00:00
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 4/4 00:03
                   all        105        417      0.787      0.835      0.878      0.651
Speed: 0.3ms pre-process, 5.3ms inference

In [3]:
%run yolov5/val.py --data fine_tune_apple_papaya_mango.yaml --weights {best_classification_weights} --img 416 --task test --single-cls

val: data=fine_tune_apple_papaya_mango.yaml, weights=['weights\\detection\\best_classification.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=True, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5\runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  2022-11-21 Python-3.10.5 torch-1.12.0 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\dataset\deepfruitvision_eval\labels.cache... 105 images, 0 backgrounds, 0 corrupt: 100%|██████████| 105/105 00:00
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 4/4 00:02
                   all        105        417       0.86        0.9      0.938       0.72
Speed: 0.3ms pre-process, 5.2ms infe

And now we can clean the Yolov5 eval dir because we don't need it any more.

In [ ]:
# also, clean up the yolov5 eval dataset
shutil.rmtree(eval_dir)

## Ensemble Classification Accuracy

Now we want to evaluate the accuracy of the ensemble classification model. Since we only care about the accuracy of the harvestability, we ignore any bounding box that is too small to be classified.

In [6]:
fruit_vision = DeepFruitVision(best_classification_weights, ripeness_weights, defect_weights, min_bounding_box_size=.05)

YOLOv5  2022-11-21 Python-3.10.5 torch-1.12.0 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [7]:
# use Matplotlib to display the true and predicted bounding boxes
# true bounding boxes are a list of dicts with x, y, w, and h keys (all normalized, not pixel values)
# predicted bounding boxes are a list of dicts with xmin, ymin, xmax, ymax (all normalized, not pixel values)
def display_bboxes(img, true_bboxes, pred_bboxes):
    img_h, img_w, _ = img.shape
    fig, ax = plt.subplots(1)
    ax.imshow(img)
    for bbox in true_bboxes:
        x = bbox['x'] * img_w
        y = bbox['y'] * img_h
        w = bbox['w'] * img_w
        h = bbox['h'] * img_h
        rect = matplotlib.patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='g', facecolor='none')
        ax.add_patch(rect)
    for bbox in pred_bboxes:
        x = bbox['xmin'] * img_w
        y = bbox['ymin'] * img_h
        w = (bbox['xmax'] - bbox['xmin']) * img_w
        h = (bbox['ymax'] - bbox['ymin']) * img_h
        rect = matplotlib.patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()

for i in range(1):
    img, true_boxes = deepfruitvision_test_dataset[i]
    pred_boxes = fruit_vision.get_harvestability(img)
    display_bboxes(img, true_boxes, pred_boxes)

In [9]:
min_box_size = fruit_vision.min_bounding_box_size

total_boxes = 0
total_correct = 0

correct_bbox = 0
correct_ripeness = 0
correct_ripeness_true_bbox = 0
correct_defect = 0

for frame, true_boxes in tqdm(deepfruitvision_test_dataset, desc='Evaluating DeepFruitVision', total=len(deepfruitvision_test_dataset)):
    predicted_boxes = fruit_vision.get_harvestability(frame)

    # ignore any boxes in the predicted and true boxes that are too small to be classified
    predicted_boxes = [box for box in predicted_boxes if box['xmax'] - box['xmin'] > min_box_size and box['ymax'] - box['ymin'] > min_box_size]
    # the true boxes are numpy arrays with the format [class, x, y, w, h, ...]
    true_boxes = [box for box in true_boxes if box['w'] > min_box_size and box['h'] > min_box_size]

    if len(true_boxes) == 0: # if there are no boxes large enough to be classified, then we can't evaluate this frame
        continue

    total_boxes += len(true_boxes)

    if len(predicted_boxes) == 0: # if there are no predicted boxes, then we automatically get 0 correct
        continue
    
    # now we have to convert the predicted and true boxes to tensors so we can use Yolo-v5's built-in IoU function
    predicted_boxes_tensor = torch.tensor([[box['xmin'], box['ymin'], box['xmax'], box['ymax']] for box in predicted_boxes])
    # each box has to have the format [x1, y1, x2, y2] where x1 < x2 and y1 < y2
    true_boxes_tensor = torch.tensor([[box['x'], box['y'], box['x'] + box['w'], box['y'] + box['h']] for box in true_boxes])

    # this returns a tensor [num_true_boxes, num_predicted_boxes] that we can use to determine which of the true boxes have a corresponding predicted box
    iou = box_iou(true_boxes_tensor, predicted_boxes_tensor)

    # the max of each row will be the IoU of the predicted box with the true box
    max_ious, max_iou_indices = torch.max(iou, dim=1)

    for i, (max_iou, max_iou_index) in enumerate(zip(max_ious, max_iou_indices)):
        if max_iou > 0.5: # if the IoU is greater than 0.5, then we consider it a correct prediction
            correct_bbox += 1

            if true_boxes[i]['ripeness'] == predicted_boxes[max_iou_index]['ripeness'][0]:
                correct_ripeness += 1

            if true_boxes[i]['defect'] == predicted_boxes[max_iou_index]['defect'][0]:
                correct_defect += 1

            true_ensemble_label = true_boxes[i]['ensemble']
            predicted_harvestability_label = predicted_boxes[max_iou_index]['harvestability']

            if true_ensemble_label == predicted_harvestability_label:
                total_correct += 1

print(f'Got {total_correct} out of {total_boxes} correct ({total_correct / total_boxes * 100:.2f}%)')
print(f'Got {correct_bbox} out of {total_boxes} correct bounding boxes ({correct_bbox / total_boxes * 100:.2f}%)')
print(f'Got {correct_ripeness} out of {total_boxes} correct ripeness labels ({correct_ripeness / total_boxes * 100:.2f}%)')
print(f'Got {correct_defect} out of {total_boxes} correct defect labels ({correct_defect / total_boxes * 100:.2f}%)')

Evaluating DeepFruitVision:   0%|          | 0/105 [00:00<?, ?it/s]

Got 327 out of 417 correct (78.42%)
Got 396 out of 417 correct bounding boxes (94.96%)
Got 340 out of 417 correct ripeness labels (81.53%)
Got 341 out of 417 correct ripeness labels when the bounding box was correct (81.77%)
Got 364 out of 417 correct defect labels (87.29%)


## Speed

The time it takes to run the model on a single frame is variable, so we will run it on all the images in the ensemble dataset and take the average of them. We will load the images into memeory first so that we don't have to deal with the time to load them from the disk.

In [7]:
import time
from modules.preprocessor import localize_fruit

ensemble_dataset.for_yolov5 = False
ensemble_images = []
localized_img_tensors = []
for frame, _ in tqdm(ensemble_dataset, desc='Loading images', total=len(ensemble_dataset)):
    ensemble_images.append(frame)
    bounding_boxes = fruit_vision.detection_module.get_bounding_boxes(frame)
    localized_imgs = localize_fruit(frame, bounding_boxes, fruit_vision.min_bounding_box_size)

    localized_img_tensors.append(localized_imgs)

Loading images:   0%|          | 0/155 [00:00<?, ?it/s]

In [8]:
start = time.time()

for frame in tqdm(ensemble_images, desc='Running model', total=len(ensemble_images)):
    fruit_vision.get_harvestability(frame)

end = time.time()

Running model:   0%|          | 0/155 [00:00<?, ?it/s]

In [9]:
print(f'Average time per frame: {(end - start) / len(ensemble_images):.2f} seconds')
print(f'Average FPS: {len(ensemble_images) / (end - start):.2f}')

Average time per frame: 0.69 seconds
Average FPS: 1.46


We achieve around 1.5 FPS on each frame, or about .66 seconds. About half of that time is spent on the deep learning models (Yolo-v5, EfficientNet-v2S for defect and ripeness detection) and the other half is spent using the bounding boxes to localize the fruit for ripeness and defect detection. The `preprocessor.localize_fruit` is unoptimized and has a lot of room for improvement.

In [19]:
start = time.time()

for frame, tensor_stack in tqdm(zip(ensemble_images, localized_img_tensors), desc='Running model', total=len(localized_img_tensors)):
    bounding_boxes = fruit_vision.detection_module.get_bounding_boxes(frame)
    fruit_vision.ripeness_module.model(tensor_stack)
    fruit_vision.defect_module.model(tensor_stack)

end = time.time()

Running model:   0%|          | 0/155 [00:00<?, ?it/s]

In [20]:
print(f'Average time per frame: {(end - start) / len(ensemble_images):.2f} seconds')
print(f'Average FPS: {len(ensemble_images) / (end - start):.2f}')

Average time per frame: 0.32 seconds
Average FPS: 3.13
